In [ ]:
import numpy as np

In [ ]:
# References:
# Kai Tao, Tianze Liu, Jieyuan Ning, Fenglin Niu, "Estimating sedimentary and crustal structure
# using wavefield continuation: theory, techniques and applications", Geophysical Journal International,
# Volume 197, Issue 1, April, 2014, Pages 443-457, https://doi.org/10.1093/gji/ggt515

In [ ]:
import sympy

In [ ]:
sympy.init_printing()

In [ ]:
sympy.var('α β p ρ μ ω', positive=True, real=True)

In [ ]:
sympy.var('z z0 t t0 t1', real=True)

In [ ]:
μ = ρ*β**2
μ

In [ ]:
q_α = sympy.sqrt(1/α**2 - p**2)
q_α

In [ ]:
q_β = sympy.sqrt(1/β**2 - p**2)
q_β

In [ ]:
η = 1/β**2 - 2*p**2
η

In [ ]:
M = sympy.Matrix([
    [         α*p,         α*p,       β*q_β,       β*q_β],
    [       α*q_α,      -α*q_α,        -β*p,         β*p],
    [-2*α*μ*p*q_α, 2*α*μ*p*q_α,      -β*μ*η,       β*μ*η],
    [      -α*μ*η,      -α*μ*η, 2*β*μ*p*q_β, 2*β*μ*p*q_β]
])
M

In [ ]:
Minv = (1/ρ)*sympy.Matrix([
    [        μ*p/α,  η*μ/(2*α*q_α), -p/(2*α*q_α),    -1/(2*α)],
    [        μ*p/α, -η*μ/(2*α*q_α),  p/(2*α*q_α),    -1/(2*α)],
    [η*μ/(2*β*q_β),         -μ*p/β,     -1/(2*β), p/(2*β*q_β)],
    [η*μ/(2*β*q_β),          μ*p/β,      1/(2*β), p/(2*β*q_β)]
])
Minv

In [ ]:
deig = sympy.diag(-q_α, q_α, -q_β, q_β)
deig

In [ ]:
# Lame constant
λ = ρ*(α**2 - 2*β**2)
λ

In [ ]:
γ = 4*μ*(λ + μ)/(λ + 2*μ)
γ

In [ ]:
A = sympy.Matrix([
    [0, p, 1/μ, 0],
    [p*λ/(λ + 2*μ), 0, 0, 1/(λ + 2*μ)],
    [ρ - (p**2)*γ, 0, 0, p*λ/(λ + 2*μ)],
    [0, ρ, p, 0]
])
A

In [ ]:
sympy.simplify(A - M*deig*Minv)

In [ ]:
Pdiag = sympy.diag(sympy.exp(sympy.I*ω*q_α*(z - z0)),
                   sympy.exp(-sympy.I*ω*q_α*(z - z0)),
                   sympy.exp(sympy.I*ω*q_β*(z - z0)),
                   sympy.exp(-sympy.I*ω*q_β*(z - z0)))
Pdiag

In [ ]:
P = M*Pdiag*Minv

## Study case of half-space with no layers (mantle only)

In [ ]:
sympy.var('v_r0 v_z0')

In [ ]:
P0 = P.subs([(z0, 0)])

In [ ]:
w = Minv*P0*sympy.Matrix([v_r0, v_z0, 0, 0])

In [ ]:
S_up = sympy.simplify(w[3])

In [ ]:
St_up = sympy.InverseFourierTransform(S_up, ω, t)

In [ ]:
St_up

In [ ]:
Jz_Sup = -ρ*(β**2)*q_β*sympy.Abs(St_up)**2

In [ ]:
Jz_Sup

In [ ]:
energy = sympy.Integral(Jz_Sup, (t, t0, t1))

In [ ]:
energy

In [ ]:
# Units of density don't matter since it turns out to be just a scaling factor

--------------------------------------------------------

# Load some data and start computing cost function

In [ ]:
from collections import defaultdict
import logging

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import numpy.fft as fft

import h5py
import obspy
import obspyh5

from seismic.receiver_fn.stream_quality_filter import curate_stream3c
from seismic.receiver_fn.rf_util import compute_vertical_snr

## Test first on non-sedimentary station since the theory is supposed to still work

In [ ]:
network = 'OA'
target_station = 'BT23'

In [ ]:
src_file = r"/g/data1a/ha3/am7399/shared/OA_RF_analysis/OA_event_waveforms_for_rf_20170911T000036-20181128T230620_rev8.h5"

In [ ]:
traces = []
for tr in obspyh5.iterh5(src_file, group='/waveforms/{}.{}.0M'.format(network, target_station), mode='r'):
    traces.append(tr)

In [ ]:
# Group triplets of traces for same event id
data_all = defaultdict(obspy.Stream)
for tr in traces:
    data_all[tr.stats.event_id].append(tr) 

In [ ]:
# Re-order traces into ZNE order
for evid, stream in data_all.items():
    stream.sort(keys=['channel'], reverse=True)

In [ ]:
len(data_all)

In [ ]:
data_all['smi:ISC/evid=611069756']

In [ ]:
# Apply curation to streams prior to rotation
logger = logging.getLogger(__name__)
for evid, stream in data_all.items():
    if not curate_stream3c(evid, stream, logger):
        data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
# Rotate to ZRT coordinates
for evid, stream in data_all.items():
    stream.rotate('NE->RT')

In [ ]:
sample_data = data_all[list(data_all.keys())[0]]
sample_data

In [ ]:
sample_data.plot()

In [ ]:
# Detrend and taper the traces
for evid, stream in data_all.items():
    stream.detrend('linear')
    stream.taper(0.05)

In [ ]:
sample_data.plot()

In [ ]:
# NOTE: We are not applying any spectral filtering to ZRT waveforms.

In [ ]:
sample_data.copy().trim(starttime=sample_data[0].stats.onset - 10,
                        endtime=sample_data[0].stats.onset + 25).plot()

In [ ]:
sample_data[0].stats

In [ ]:
# Compute SNR of Z component to use as a quality metric
for evid, stream in data_all.items():
    compute_vertical_snr(stream)

In [ ]:
sample_data[0].stats

In [ ]:
snrs = np.array([s[0].stats.snr_prior for _, s in data_all.items()])

In [ ]:
plt.hist(snrs, bins=np.linspace(0, 10, 21))
plt.show()

In [ ]:
discard_ids = []
for evid, stream in data_all.items():
    if stream[0].stats.snr_prior < 2.0:
        discard_ids.append(evid)
        
for evid in discard_ids:
    data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
sample_data = data_all[list(data_all.keys())[0]]
sample_data.plot()